In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)

# Create our target
y = df.loc[:,target].copy()

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [8]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=100)
X_train.shape
y_train.shape

(51612, 1)

In [10]:
X_train


,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
13143,10000.0,0.0646,306.31,80000.0,13.56,0.0,0.0,11.0,1.0,3879.0,...,0,1,0,1,1,0,1,0,1,1
25882,15000.0,0.0646,459.47,190000.0,9.61,0.0,0.0,23.0,0.0,24055.0,...,0,1,0,1,0,1,1,0,1,1
55417,27600.0,0.1298,929.69,150000.0,19.77,0.0,2.0,13.0,0.0,31513.0,...,0,1,0,1,0,1,1,0,1,1
42457,19000.0,0.1180,629.26,50000.0,35.74,0.0,0.0,10.0,0.0,16934.0,...,0,1,0,1,1,0,1,0,1,1
62360,18000.0,0.1131,394.16,250000.0,4.29,0.0,0.0,12.0,1.0,25682.0,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14260,7200.0,0.0881,228.33,76000.0,18.95,0.0,1.0,9.0,0.0,13644.0,...,0,1,0,1,1,0,1,0,1,1
63370,30000.0,0.1180,664.31,37000.0,51.99,0.0,0.0,7.0,0.0,4053.0,...,0,1,0,1,0,1,0,1,1,1
65615,16000.0,0.2534,472.82,28000.0,24.69,0.0,0.0,8.0,0.0,1857.0,...,0,1,0,1,0,1,1,0,1,1
56088,5600.0,0.1797,202.37,85000.0,28.63,0.0,2.0,24.0,1.0,17749.0,...,0,1,0,1,0,1,1,0,1,1


In [11]:
y_train

,loan_status
13143,low_risk
25882,low_risk
55417,low_risk
42457,low_risk
62360,low_risk
...,...
14260,low_risk
63370,low_risk
65615,low_risk
56088,low_risk


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brfc_model = BalancedRandomForestClassifier(n_estimators = 100, random_state=1)
brfc_model.fit(X_train,y_train)

BalancedRandomForestClassifier(random_state=1)

In [14]:
y_pred=brfc_model.predict(X_test)

In [15]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test,y_pred)

0.7573208743728974

In [16]:

# Display the confusion matrix
#y_pred = rf_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   49,    29],
       [ 1945, 15182]], dtype=int64)

In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.63      0.89      0.05      0.75      0.54        78
   low_risk       1.00      0.89      0.63      0.94      0.75      0.57     17127

avg / total       0.99      0.89      0.63      0.93      0.75      0.57     17205



In [18]:
# List the features sorted in descending order by feature importance
importances = sorted(zip(brfc_model.feature_importances_, X.columns), reverse = True)

importances

[(0.08438109712038916, 'total_rec_prncp'),
 (0.06023157514280019, 'total_rec_int'),
 (0.059652086285770706, 'last_pymnt_amnt'),
 (0.054188517505757594, 'total_pymnt'),
 (0.051845956847915056, 'total_pymnt_inv'),
 (0.03189988089521828, 'issue_d_Jan-2019'),
 (0.030519103949698482, 'int_rate'),
 (0.019088730184821056, 'dti'),
 (0.017879496692673535, 'annual_inc'),
 (0.01761092439953712, 'installment'),
 (0.016147663703051023, 'mo_sin_old_rev_tl_op'),
 (0.016068106569389333, 'bc_open_to_buy'),
 (0.015897386747954594, 'max_bal_bc'),
 (0.015228420058174244, 'mths_since_recent_inq'),
 (0.014698658035887836, 'tot_hi_cred_lim'),
 (0.014538645416659382, 'revol_bal'),
 (0.01444008011455452, 'mo_sin_old_il_acct'),
 (0.014363383986905148, 'avg_cur_bal'),
 (0.014284021680335162, 'il_util'),
 (0.014168997511574487, 'total_bal_ex_mort'),
 (0.014053040281548277, 'out_prncp'),
 (0.013936304253280069, 'total_bc_limit'),
 (0.01371339693072474, 'out_prncp_inv'),
 (0.01353312504198071, 'total_rev_hi_lim'),


### Easy Ensemble AdaBoost Classifier

In [19]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state = 1)

eec.fit(X_train, y_train)


EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [20]:
X_train.shape
y_train.shape

(51612, 1)

In [21]:
# Calculated the balanced accuracy score
y_pred=eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9088932903961806

In [22]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   68,    10],
       [  925, 16202]], dtype=int64)

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.87      0.95      0.13      0.91      0.82        78
   low_risk       1.00      0.95      0.87      0.97      0.91      0.83     17127

avg / total       1.00      0.95      0.87      0.97      0.91      0.83     17205

